# ИНСТРУКЦИЯ
перед запуском кода необходимо загрузить в среду файл project_update-osnova.py

In [1]:
!pip install streamlit pyngrok --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 125.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.1 MB/s eta 0:00:00


In [2]:
!ngrok config add-authtoken 2xrRRZKJrMVN1qYzB9gfkaVX0qr_uk1jP9YuiwXNrMVHP9qz

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import zipfile
import shutil

# ЗАГРУЗКА ПРЕДОБУЧЕННЫХ МОДЕЛЕЙ НА ДИСК
необходимо разрешить доступ к диску (вылезет соответствующая плашка)

In [4]:
try:
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
except Exception as auth_error:
    print(f"Ошибка авторизации: {auth_error}")
    print("Попробуйте: Runtime -> Restart runtime и запустите снова")
    raise

def download_and_extract_from_gdrive(file_id, zip_name):

    try:

        print(f"⏳ Скачиваем файл '{zip_name}'...")
        downloaded = drive.CreateFile({'id': file_id})
        downloaded.GetContentFile(zip_name)


        folder_name = os.path.splitext(zip_name)[0]
        target_folder = f'/content/{folder_name}'


        temp_folder = '/content/temp_extract'
        os.makedirs(temp_folder, exist_ok=True)


        print(f"📦 Распаковываем архив...")
        with zipfile.ZipFile(zip_name, 'r') as zip_ref:
            zip_ref.extractall(temp_folder)


        extracted_items = os.listdir(temp_folder)


        if len(extracted_items) == 1 and os.path.isdir(os.path.join(temp_folder, extracted_items[0])):
            inner_folder = os.path.join(temp_folder, extracted_items[0])


            shutil.move(inner_folder, target_folder)
        else:

            os.rename(temp_folder, target_folder)


        os.remove(zip_name)
        try:
            os.rmdir(temp_folder)
        except OSError:
            pass

        print(f"✅ Готово! Модель распакована в: {target_folder}")

        return target_folder

    except Exception as e:
        print(f"❌ Ошибка: {str(e)}")
        if 'zip_name' in locals() and os.path.exists(zip_name):
            os.remove(zip_name)
        if 'temp_folder' in locals() and os.path.exists(temp_folder):
            shutil.rmtree(temp_folder, ignore_errors=True)
        raise

if __name__ == "__main__":
    # Список моделей для загрузки
    models_to_download = [
        ("1gpEKpLYoMJV7ui-Hy9BKXKJnWAdXcFIL", "gazeta_summarizer_mT5_m2o_russian_crossSum.zip"),
        ("1BxySSxAeXOer8WHerCGQ5C3h4VdQ8zKt", "gazeta_summarizer_dorj.zip"),
        ("1syLs1V5k5uB8Ay_6sluYqN-oKbdqSsKz", "gazeta_summarizer_sber_base.zip")
    ]

    for file_id, zip_name in models_to_download:
        print(f"\n{'='*50}")
        print(f"Начинаем загрузку модели: {zip_name}")
        print(f"{'='*50}")

        try:
            model_path = download_and_extract_from_gdrive(file_id, zip_name)
            print(f"Модель {zip_name} успешно загружена и распакована в {model_path}")
        except Exception as e:
            print(f"Не удалось загрузить модель {zip_name}. Ошибка: {e}")
            continue


Начинаем загрузку модели: gazeta_summarizer_mT5_m2o_russian_crossSum.zip
⏳ Скачиваем файл 'gazeta_summarizer_mT5_m2o_russian_crossSum.zip'...
📦 Распаковываем архив...
✅ Готово! Модель распакована в: /content/gazeta_summarizer_mT5_m2o_russian_crossSum
Модель gazeta_summarizer_mT5_m2o_russian_crossSum.zip успешно загружена и распакована в /content/gazeta_summarizer_mT5_m2o_russian_crossSum

Начинаем загрузку модели: gazeta_summarizer_dorj.zip
⏳ Скачиваем файл 'gazeta_summarizer_dorj.zip'...
📦 Распаковываем архив...
✅ Готово! Модель распакована в: /content/gazeta_summarizer_dorj
Модель gazeta_summarizer_dorj.zip успешно загружена и распакована в /content/gazeta_summarizer_dorj

Начинаем загрузку модели: gazeta_summarizer_sber_base.zip
⏳ Скачиваем файл 'gazeta_summarizer_sber_base.zip'...
📦 Распаковываем архив...
✅ Готово! Модель распакована в: /content/gazeta_summarizer_sber_base
Модель gazeta_summarizer_sber_base.zip успешно загружена и распакована в /content/gazeta_summarizer_sber_base

In [7]:
from pyngrok import ngrok
import threading
import time
import os

# Запуск Streamlit
def run_streamlit():
    os.system('streamlit run project_update-osnova.py')

thread = threading.Thread(target=run_streamlit)
thread.start()


time.sleep(3)

# туннель на порт 8501
public_url = ngrok.connect(8501)
print(f"🌍 Открой в браузере: {public_url}")

🌍 Открой в браузере: NgrokTunnel: "https://3a0b-34-16-174-78.ngrok-free.app" -> "http://localhost:8501"
